In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import tqdm
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,image_id,labels
0,100241706_00004_2,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...
1,100241706_00005_1,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...
2,100241706_00005_2,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...
3,100241706_00006_1,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...
4,100241706_00007_2,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...


In [4]:
sample = train.labels[0]

In [5]:
sample

'U+306F 1231 3465 133 53 U+304C 275 1652 84 69 U+3044 1495 1218 143 69 U+3051 220 3331 53 91 U+306B 911 1452 61 92 U+306B 927 3445 71 92 U+306E 904 2879 95 92 U+5DE5 1168 1396 187 95 U+3053 289 3166 69 97 U+4E09 897 3034 121 107 U+306E 547 1912 141 108 U+3084 1489 2675 151 109 U+3068 1561 2979 55 116 U+5DF1 1513 2500 127 117 U+3082 1213 1523 72 119 U+3055 1219 3266 95 124 U+306E 259 2230 68 125 U+306E 1184 2423 169 125 U+4E16 849 2236 163 127 U+7D30 1144 1212 200 128 U+305D 316 3287 57 133 U+4EBA 217 2044 183 135 U+3051 277 2974 112 137 U+308C 201 3423 181 137 U+3060 243 2830 159 143 U+5F37 1479 2034 163 145 U+306E 1497 1567 123 152 U+305F 1164 952 145 153 U+3066 552 1199 97 155 U+4FF3 537 2095 176 155 U+6839 203 1439 184 156 U+304B 1188 2606 156 157 U+8AE7 549 2328 156 159 U+308C 1495 2784 168 159 U+5B50 891 1255 100 164 U+3092 584 2546 117 164 U+53CA 849 1588 151 164 U+8005 1192 2198 133 169 U+305A 889 1763 103 171 U+907F 513 945 181 171 U+6B63 539 1439 136 172 U+6587 192 2382 216 17

In [6]:
def split_labels(sample):
    
    counter = 0
    
    class_type = []
    xmin = []
    ymin = []
    xmax = []
    ymax = []

    for i in sample.split():
        if counter == 0:
            class_type.append(i)
            counter += 1

        elif counter == 1:
            xmin.append(int(i))
            counter += 1

        elif counter == 2:
            ymin.append(int(i))
            counter += 1

        elif counter == 3:
            xmax.append(int(i) + xmin[-1])
            counter += 1

        elif counter == 4:
            ymax.append(int(i) + ymin[-1])
            counter = 0
    
    return class_type, xmin, xmax, ymin, ymax

In [7]:
train.fillna('', inplace=True)

In [8]:
train_path = '/home/stealth/Documents/models/research/kuzusiji/data/train/'

In [9]:
new_train = []
for i, r in tqdm.tqdm_notebook(train.iterrows()):
    
    im = Image.open(train_path+r['image_id']+'.jpg')
    
    class_label, xmin, xmax, ymin, ymax = split_labels(r['labels']);
    
    image_id = [r['image_id']] * len(class_label)
    width = [im.width] * len(class_label)
    height = [im.height] * len(class_label)
    
    new_train.append(pd.DataFrame(list(zip(image_id, width, height, class_label, xmin, xmax, ymin, ymax)),
                                  columns=['image_id', 'width', 'height', 'class',
                                           'xmin', 'xmax', 'ymin', 'ymax']))

In [10]:
new_train = pd.concat(new_train).reset_index(drop=True)

In [11]:
train.tail()

,image_id,labels
3876,umgy012-039,U+309D 426 456 30 34 U+306F 584 359 82 45 U+30...
3877,umgy012-040,U+30DE 677 327 68 35 U+3078 1424 463 110 43 U+...
3878,umgy012-041,U+309D 1582 802 16 34 U+306F 597 2337 72 42 U+...
3879,umgy012-042,U+4E00 1050 898 86 21 U+309D 724 1864 27 29 U+...
3880,umgy012-043,


In [12]:
new_train[new_train['image_id'] == 'umgy012-042']

,image_id,width,height,class,xmin,xmax,ymin,ymax
683308,umgy012-042,1768,2938,U+4E00,1050,1136,898,919
683309,umgy012-042,1768,2938,U+309D,724,751,1864,1893
683310,umgy012-042,1768,2938,U+309D,533,558,902,938
683311,umgy012-042,1768,2938,U+306F,1447,1526,1869,1913
683312,umgy012-042,1768,2938,U+306F,1072,1144,448,494
683313,umgy012-042,1768,2938,U+4E8C,119,224,1514,1562
683314,umgy012-042,1768,2938,U+3079,506,593,2219,2268
683315,umgy012-042,1768,2938,U+304A,864,958,927,977
683316,umgy012-042,1768,2938,U+304A,1062,1150,507,558
683317,umgy012-042,1768,2938,U+4E8C,1052,1156,1465,1516


In [13]:
new_train.to_csv('new_train.csv', index=False)

In [14]:
!ls -al

total 3807984
drwxr-xr-x  4 stealth stealth       4096 Jul 21 22:27 .
drwxr-xr-x 71 stealth stealth       4096 Jul 21 20:42 ..
drwxr-xr-x  4 stealth stealth       4096 Jul 21 22:17 data
-rw-r--r--  1 stealth stealth      37968 Jul 21 22:23 data_prep.ipynb
drwxr-xr-x  2 stealth stealth       4096 Jul 21 20:44 .ipynb_checkpoints
-rw-r--r--  1 stealth stealth   35191033 Jul 21 22:27 new_train.csv
-rw-rw-r--  1 stealth stealth 3848386548 Jul 21 07:39 test_images.zip
-rw-------  1 stealth stealth   15677207 Jul 16 21:52 train.csv
-rw-rw-r--  1 stealth stealth      52646 Jul 21 15:58 unicode_translation.csv
